# Template Network Definitions

<i>Version 2</i>

## References

1. ["Maintaining Knowledge about Temporal Intervals" by James F. Allen](https://cse.unl.edu/~choueiry/Documents/Allen-CACM1983.pdf) - Allen's original paper (PDF)
1. [Allen's Interval Algebra](https://www.ics.uci.edu/~alspaugh/cls/shr/allen.html) or [here](https://thomasalspaugh.org/pub/fnd/allen.html) - summarizes Allen's algebra of proper time intervals
1. [W3C Time Ontology in OWL](https://www.w3.org/TR/owl-time/) - temporal vocabulary used here is based on the W3C vocabulary of time
1. [bitsets Python package](https://bitsets.readthedocs.io/en/stable/) - used to implement Algebra relation sets and operations
1. [NetworkX Python package](http://networkx.github.io/) - used to represent directed graph of constraints
1. [Python format string syntax](https://docs.python.org/3/library/string.html#format-string-syntax) - used in Algebra summary method
1. [Spatial Ontology](https://www.w3.org/2017/sdwig/bp/) - I'm still looking for a standard spatial vocabulary; maybe start here
1. [Qualitative Spatial Relations (QSR) Library](https://qsrlib.readthedocs.io/en/latest/index.html) - an alternative library to the one defined here

## Dependencies

In [1]:
import os
import qualreas as qr
import numpy as np

In [2]:
path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

In [3]:
#pt_alg = qr.Algebra(os.path.join(path, "Algebras/LinearPointAlgebra.json"))
#pt_alg = qr.Algebra(os.path.join(path, "Algebras/RightBranchingPointAlgebra.json"))
pt_alg = qr.Algebra(os.path.join(path, "Algebras/LeftBranchingPointAlgebra.json"))

## Create Algebra Elements using 4-Point Networks

### Define 4-Point Network Generator

In [5]:
foobar = qr.FourPoint(pt_alg, "Foobar", "<|=")
foobar.summary()


Foobar: 4 nodes, 8 edges
  Algebra: LeftBranchingPointAlgebra
  StartPt1:
    => StartPt1: =
    => EndPt1: <|=
  EndPt1:
    => EndPt1: =
    => StartPt1: =|>
  StartPt2:
    => StartPt2: =
    => EndPt2: <|=
  EndPt2:
    => EndPt2: =
    => StartPt2: =|>


In [19]:
# Viewing the network as a matrix, 'elem13', below, refers to the element in row 1 col 3,
# and so on for 'elem23', etc.  The matrix is 4x4, so if we partition it into four 2x2
# matrices, then the two partiions on the diagonal represent two intervals and the two
# off-diagonal partitions represent how those two intervals relate to each other.
# Also, the off-diagonal 2x2 partitions are converse transposes of each other.
# Oh, and the intervals represented by the diagonal partitions could be intervals,
# proper intervals, or points.

def generate_consistent_networks(alg, lessthan="<", startname="StartPt", endname="EndPt",
                                 verbose=False):
    consistent_nets = dict()
    for elem13 in alg.elements:
        for elem23 in alg.elements:
            for elem14 in alg.elements:
                for elem24 in alg.elements:
                    four_pt_net_name = elem13 + ',' + elem23 + ',' + elem14 + ',' + elem24
                    #net, pts = four_point_network(alg, lessthan, startname, endname)
                    net = qr.FourPoint(alg, four_pt_net_name, lessthan, startname, endname)
                    #pt1, pt2, pt3, pt4 = pts
                    pt1, pt2, pt3, pt4 = net.get_points()
                    rs13 = alg.relset(elem13)
                    rs23 = alg.relset(elem23)
                    rs14 = alg.relset(elem14)
                    rs24 = alg.relset(elem24)
                    net.add_constraint(pt1, pt3, rs13)
                    net.add_constraint(pt2, pt3, rs23)
                    net.add_constraint(pt1, pt4, rs14)
                    net.add_constraint(pt2, pt4, rs24)
                    if net.propagate():
                        elem_key = ",".join([str(rs13), str(rs14), str(rs23), str(rs24)])
                        consistent_nets[qr.key_name_mapping[elem_key]] = net
                        if verbose:
                            print("==========================")
                            if elem_key in qr.key_name_mapping:
                                print(qr.key_name_mapping[elem_key])
                            else:
                                print("UNKNOWN")
                            #print(np.matrix(constraint_matrix_to_list(net, pts)))
                            print(np.matrix(net.to_list()))
    print(f"\n{len(consistent_nets)} consistent networks")
    return consistent_nets

### Generating a 4-Point Network that Represents 2 Intervals

In [20]:
#net4pt = four_point_network(pt_alg, "=|<")
net4pt = qr.FourPoint(pt_alg, "FourPointNetwork", "=|<")
pts = net4pt.get_points()
net4pt.summary()


FourPointNetwork: 4 nodes, 8 edges
  Algebra: LeftBranchingPointAlgebra
  StartPt1:
    => StartPt1: =
    => EndPt1: <|=
  EndPt1:
    => EndPt1: =
    => StartPt1: =|>
  StartPt2:
    => StartPt2: =
    => EndPt2: <|=
  EndPt2:
    => EndPt2: =
    => StartPt2: =|>


In [21]:
for pt in pts:
    print(pt)

<TemporalEntity StartPt1 ['Point']>
<TemporalEntity EndPt1 ['Point']>
<TemporalEntity StartPt2 ['Point']>
<TemporalEntity EndPt2 ['Point']>


## Derive Algebra Elements

A 4-point network, like that generated above, only has constraints specified so that the first two points define an interval, and same for the second two points.  No constraints are specified between the two implied intervals (e.g., no constraint between StartPt1/EndPt1 and StartPt2/EndPt2).  Depending on which point algebra is used there are either 3^4 (81) or 4^4 (256) different ways the unassigned constraint pairs can be made.  The function,  <i>generate_consistent_networks</i> tries all of these possibilities and returns the ones that are consistent.  Doing this for the linear point algebra ('<', '=', '>') results in 13 consistent networks that correspond to Allen's Temporal Algebra of Proper Time Intervals.  Using ('<|=', '=', '>|=") results in 18 consistent networks that are a superset of Allen's relations that includes 5 additional relations that integrate points into the algebra.  Using ('<|=', '=', '>|=', '~'), where '~' is either the left-incomparable or right-incomparable relation of the left- or right-branching time point algebra will result in 24 consistent networks that integrate points into a left- or right-branching time interval algebra.

In [22]:
consistent_nets = generate_consistent_networks(pt_alg, lessthan="=|<", verbose=True)

B
[['=' '<|=' '<' '<']
 ['=|>' '=' '<' '<']
 ['>' '>' '=' '<|=']
 ['>' '>' '=|>' '=']]
M
[['=' '<' '<' '<']
 ['>' '=' '=' '<']
 ['>' '=' '=' '<']
 ['>' '>' '>' '=']]
PFI
[['=' '<' '<' '<']
 ['>' '=' '=' '=']
 ['>' '=' '=' '=']
 ['>' '=' '=' '=']]
O
[['=' '<' '<' '<']
 ['>' '=' '>' '<']
 ['>' '<' '=' '<']
 ['>' '>' '>' '=']]
FI
[['=' '<' '<' '<']
 ['>' '=' '>' '=']
 ['>' '<' '=' '<']
 ['>' '=' '>' '=']]
DI
[['=' '<' '<' '<']
 ['>' '=' '>' '>']
 ['>' '<' '=' '<|=']
 ['>' '<' '=|>' '=']]
PS
[['=' '=' '=' '<']
 ['=' '=' '=' '<']
 ['=' '=' '=' '<']
 ['>' '>' '>' '=']]
PE
[['=' '=' '=' '=']
 ['=' '=' '=' '=']
 ['=' '=' '=' '=']
 ['=' '=' '=' '=']]
S
[['=' '<' '=' '<']
 ['>' '=' '>' '<']
 ['=' '<' '=' '<']
 ['>' '>' '>' '=']]
E
[['=' '<' '=' '<']
 ['>' '=' '>' '=']
 ['=' '<' '=' '<']
 ['>' '=' '>' '=']]
SI
[['=' '<' '=' '<']
 ['>' '=' '>' '>']
 ['=' '<' '=' '<']
 ['>' '<' '>' '=']]
PSI
[['=' '<' '=' '=']
 ['>' '=' '>' '>']
 ['=' '<' '=' '=']
 ['=' '<' '=' '=']]
D
[['=' '<|=' '>' '<']
 ['=|>' 

In [23]:
consistent_nets

{'B': <qualreas.FourPoint at 0x7f8d291eada0>,
 'M': <qualreas.FourPoint at 0x7f8d1853d4a8>,
 'PFI': <qualreas.FourPoint at 0x7f8d291e3160>,
 'O': <qualreas.FourPoint at 0x7f8d386203c8>,
 'FI': <qualreas.FourPoint at 0x7f8d18542dd8>,
 'DI': <qualreas.FourPoint at 0x7f8d18542780>,
 'PS': <qualreas.FourPoint at 0x7f8d38629f98>,
 'PE': <qualreas.FourPoint at 0x7f8d3862af98>,
 'S': <qualreas.FourPoint at 0x7f8d3862cf98>,
 'E': <qualreas.FourPoint at 0x7f8d3862ceb8>,
 'SI': <qualreas.FourPoint at 0x7f8d38630dd8>,
 'PSI': <qualreas.FourPoint at 0x7f8d386357f0>,
 'D': <qualreas.FourPoint at 0x7f8d3863ff98>,
 'F': <qualreas.FourPoint at 0x7f8d38630eb8>,
 'OI': <qualreas.FourPoint at 0x7f8d38641f28>,
 'PF': <qualreas.FourPoint at 0x7f8d386436d8>,
 'MI': <qualreas.FourPoint at 0x7f8d38643f28>,
 'BI': <qualreas.FourPoint at 0x7f8d38649940>,
 'LO': <qualreas.FourPoint at 0x7f8d38653fd0>,
 'LF': <qualreas.FourPoint at 0x7f8d38643978>,
 'LOI': <qualreas.FourPoint at 0x7f8d386575c0>,
 'LBI': <qualreas

In [25]:
#def print_as_matrix(net, entities=None):
    

In [27]:
before = consistent_nets['B']
print(np.matrix(before.to_list()))

[['=' '<|=' '<' '<']
 ['=|>' '=' '<' '<']
 ['>' '>' '=' '<|=']
 ['>' '>' '=|>' '=']]


In [28]:
def ontology_classes(net, start, end):
    """The constraints between the start and end points of a temporal entity
    determine whether it belongs to the class of Point, ProperIntervals, or
    both.  Return a list containing the class names for the input network, net."""
    class_list = []
    constr = net.edges[net.get_entity(start),
                       net.get_entity(end)]['constraint']
    if '=' in constr:
        class_list.append('Point')
    if '<' in constr:
        class_list.append('ProperInterval')
    return class_list

def domain_and_range(four_point_network, startname="StartPt", endname="EndPt"):
    """Return a tuple, (domain, range), for the interval/point relation
    represented by the input 4-point network."""
    return (ontology_classes(four_point_network, startname+"1", endname+"1"),
            ontology_classes(four_point_network, startname+"2", endname+"2"))

In [34]:
for rel in consistent_nets:
    print(f"{rel}: {domain_and_range(consistent_nets[rel])}")

B: (['Point', 'ProperInterval'], ['Point', 'ProperInterval'])
M: (['ProperInterval'], ['ProperInterval'])
PFI: (['ProperInterval'], ['Point'])
O: (['ProperInterval'], ['ProperInterval'])
FI: (['ProperInterval'], ['ProperInterval'])
DI: (['ProperInterval'], ['Point', 'ProperInterval'])
PS: (['Point'], ['ProperInterval'])
PE: (['Point'], ['Point'])
S: (['ProperInterval'], ['ProperInterval'])
E: (['ProperInterval'], ['ProperInterval'])
SI: (['ProperInterval'], ['ProperInterval'])
PSI: (['ProperInterval'], ['Point'])
D: (['Point', 'ProperInterval'], ['ProperInterval'])
F: (['ProperInterval'], ['ProperInterval'])
OI: (['ProperInterval'], ['ProperInterval'])
PF: (['Point'], ['ProperInterval'])
MI: (['ProperInterval'], ['ProperInterval'])
BI: (['Point', 'ProperInterval'], ['Point', 'ProperInterval'])
LO: (['ProperInterval'], ['ProperInterval'])
LF: (['ProperInterval'], ['ProperInterval'])
LOI: (['ProperInterval'], ['ProperInterval'])
LBI: (['ProperInterval'], ['Point', 'ProperInterval'])
LB: 

In [38]:
for rel in consistent_nets:
    print(f"{rel}: {consistent_nets[rel].name}")

B: <,<,<,<
M: <,=,<,<
PFI: <,=,<,=
O: <,>,<,<
FI: <,>,<,=
DI: <,>,<,>
PS: =,=,<,<
PE: =,=,=,=
S: =,>,<,<
E: =,>,<,=
SI: =,>,<,>
PSI: =,>,=,>
D: >,>,<,<
F: >,>,<,=
OI: >,>,<,>
PF: >,>,=,=
MI: >,>,=,>
BI: >,>,>,>
LO: l~,>,<,<
LF: l~,>,<,=
LOI: l~,>,<,>
LBI: l~,>,l~,>
LB: l~,l~,<,<
L~: l~,l~,l~,l~
